# Satria Data Hitam

# Deskripsi Projek

- Untuk keperluan mengefektifkan pengelolaan anggaran BPJS Kesehatan, maka pada lomba BDC Satria Data 2022 Anda ditantang untuk melakukan prediksi status pulang peserta berdasarkan data FKTP BPJS Kesehatan. 
- Bangun pemodelan untuk klasifikasi status pulang peserta menjadi “Sehat” dan “Belum_Sehat” dengan memanfaatkan data train. 


In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imbpipeline
import xgboost as xgb


import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)

In [2]:
col = ['Nomor peserta', 'Nomor keluarga', 'Bobot', 'ID Kunjungan FKTP',
        'Tanggal datang kunjungan FKTP', 'Tanggal pulang kunjungan FKTP',
        'Provinsi FKTP', 'Kabupaten/Kota FKTP', 'Kepemilikan FKTP',
        'Jenis FKTP', 'Tipe FKTP', 'Tingkat Pelayanan FKTP', 'Jenis Poli FKTP',
        'Segmen Peserta saat akses layanan FKTP', 'Kode dan nama diagnosis ICD 10 (3 digit)',
        'Kode diagnosis ICD 10 (3 digit)', 'Kode diagnosis (beragam 3-5 digit)',
        'Nama diagnosis berasal dari kode diagnosis FKP15', 'Provinsi faskes tujuan rujukan',
        'Kabupaten/Kota faskes tujuan rujukan', 'Kepemilikan faskes tujuan rujukan', 
        'Jenis faskes tujuan rujukan', 'Tipe faskes tujuan rujukan', 
        'Poli faskes tujuan rujukan', 'Jenis Kunjungan FKTP', 'Kelas Status Pulang Peserta']

In [3]:
train = pd.read_csv(r'..\Satria-Data-BDC\train_fktp.txt', names=col, infer_datetime_format=True, parse_dates=[4,5])
test = pd.read_csv(r'..\Satria-Data-BDC\test_fktp.txt', names=col[:-1], infer_datetime_format=True, parse_dates=[4,5])

# Feature Engineering

In [4]:
train1 = train.copy()
test1 = test.copy()

clean_data = [train1, test1]

## Exploratory Data Analysis

In [5]:
print(f'Dimensi Train:\n{train.shape}')
print('-'*10)
print(f'Dimensi Test:\n{test.shape}')

Dimensi Train:
(4056898, 26)
----------
Dimensi Test:
(1014225, 25)


In [6]:
train.info(show_counts=True)
print('-'*10)
train.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4056898 entries, 0 to 4056897
Data columns (total 26 columns):
 #   Column                                            Non-Null Count    Dtype         
---  ------                                            --------------    -----         
 0   Nomor peserta                                     4056898 non-null  float64       
 1   Nomor keluarga                                    4056898 non-null  int64         
 2   Bobot                                             4056898 non-null  float64       
 3   ID Kunjungan FKTP                                 4056898 non-null  object        
 4   Tanggal datang kunjungan FKTP                     4056898 non-null  datetime64[ns]
 5   Tanggal pulang kunjungan FKTP                     4056898 non-null  datetime64[ns]
 6   Provinsi FKTP                                     4056898 non-null  int64         
 7   Kabupaten/Kota FKTP                               4056898 non-null  int64         
 8   Ke

,Nomor peserta,Nomor keluarga,Bobot,ID Kunjungan FKTP,Tanggal datang kunjungan FKTP,Tanggal pulang kunjungan FKTP,Provinsi FKTP,Kabupaten/Kota FKTP,Kepemilikan FKTP,Jenis FKTP,Tipe FKTP,Tingkat Pelayanan FKTP,Jenis Poli FKTP,Segmen Peserta saat akses layanan FKTP,Kode dan nama diagnosis ICD 10 (3 digit),Kode diagnosis ICD 10 (3 digit),Kode diagnosis (beragam 3-5 digit),Nama diagnosis berasal dari kode diagnosis FKP15,Provinsi faskes tujuan rujukan,Kabupaten/Kota faskes tujuan rujukan,Kepemilikan faskes tujuan rujukan,Jenis faskes tujuan rujukan,Tipe faskes tujuan rujukan,Poli faskes tujuan rujukan,Jenis Kunjungan FKTP,Kelas Status Pulang Peserta
163935,67915621.0,67915621,5.042509,360850320P000133,2020-03-23,2020-03-23,63,6302,3,1,4,1,1.0,1,1736,Z00,Z00,General examination and investigation of perso...,98,9998,98,98,98,98,1,Belum_Sehat
2744561,13629207.0,13656894,33.616726,301681019P000253,2019-10-14,2019-10-14,35,3514,9,2,1,1,1.0,1,344,E11,E11,Non-insulin-dependent diabetes mellitus,98,9998,98,98,98,98,1,Belum_Sehat
306698,61414029.0,154694109,29.099478,58490820P000110,2020-08-13,2020-08-13,21,2103,3,1,3,1,1.0,2,1738,Z02,Z028,Other examinations for administrative purposes,98,9998,98,98,98,98,1,Belum_Sehat


In [7]:
print(f'Jumlah sample duplicated dalam Train:\n{train1.duplicated().sum()}')
print('-'*10)
print(f'Jumlah sample duplicated dalam Test:\n{test1.duplicated().sum()}')

Jumlah sample duplicated dalam Train:
0
----------
Jumlah sample duplicated dalam Test:
0


In [8]:
print(f'Jumlah null values pada setiap kolom di Train:\n{train1.isnull().sum()}')
print('-'*10)
print(f'Jumlah null values pada setiap kolom di Test:\n{test.isnull().sum()}')

Jumlah null values pada setiap kolom di Train:
Nomor peserta                                             0
Nomor keluarga                                            0
Bobot                                                     0
ID Kunjungan FKTP                                         0
Tanggal datang kunjungan FKTP                             0
Tanggal pulang kunjungan FKTP                             0
Provinsi FKTP                                             0
Kabupaten/Kota FKTP                                       0
Kepemilikan FKTP                                          0
Jenis FKTP                                                0
Tipe FKTP                                                 0
Tingkat Pelayanan FKTP                                    0
Jenis Poli FKTP                                           1
Segmen Peserta saat akses layanan FKTP                    0
Kode dan nama diagnosis ICD 10 (3 digit)                  0
Kode diagnosis ICD 10 (3 digit)                     1

In [9]:
for i in col[:-1]:
    print(f'Jumlah unique value kolom {i} pada Test: {len(test[i].unique())}')

Jumlah unique value kolom Nomor peserta pada Test: 478035
Jumlah unique value kolom Nomor keluarga pada Test: 298556
Jumlah unique value kolom Bobot pada Test: 37316
Jumlah unique value kolom ID Kunjungan FKTP pada Test: 1014225
Jumlah unique value kolom Tanggal datang kunjungan FKTP pada Test: 731
Jumlah unique value kolom Tanggal pulang kunjungan FKTP pada Test: 735
Jumlah unique value kolom Provinsi FKTP pada Test: 34
Jumlah unique value kolom Kabupaten/Kota FKTP pada Test: 505
Jumlah unique value kolom Kepemilikan FKTP pada Test: 9
Jumlah unique value kolom Jenis FKTP pada Test: 6
Jumlah unique value kolom Tipe FKTP pada Test: 9
Jumlah unique value kolom Tingkat Pelayanan FKTP pada Test: 3
Jumlah unique value kolom Jenis Poli FKTP pada Test: 26
Jumlah unique value kolom Segmen Peserta saat akses layanan FKTP pada Test: 5
Jumlah unique value kolom Kode dan nama diagnosis ICD 10 (3 digit) pada Test: 1663
Jumlah unique value kolom Kode diagnosis ICD 10 (3 digit) pada Test: 1663
Jumlah

## Data Cleaning & Feature Engineering

In [10]:
for dataset in clean_data:
    dataset.loc[dataset['Provinsi faskes tujuan rujukan'] != 98, 'Dirujuk'] = 1
    dataset.loc[dataset['Provinsi faskes tujuan rujukan'] == 98, 'Dirujuk'] = 0
    dataset['Lama kunjungan FKTP'] = dataset['Tanggal pulang kunjungan FKTP']-dataset['Tanggal datang kunjungan FKTP']

    drop_col = ['Nomor peserta', 'Nomor keluarga', 'Bobot', 'ID Kunjungan FKTP', 'Kode diagnosis ICD 10 (3 digit)', 'Kode diagnosis (beragam 3-5 digit)', 'Nama diagnosis berasal dari kode diagnosis FKP15', 'Provinsi faskes tujuan rujukan', 'Kabupaten/Kota faskes tujuan rujukan', 'Kepemilikan faskes tujuan rujukan', 'Jenis faskes tujuan rujukan', 'Tipe faskes tujuan rujukan', 'Poli faskes tujuan rujukan', 'Tanggal datang kunjungan FKTP', 'Tanggal pulang kunjungan FKTP']

    dataset.drop(drop_col, axis=1, inplace=True)
    dataset.dropna()

In [11]:
train1

,Provinsi FKTP,Kabupaten/Kota FKTP,Kepemilikan FKTP,Jenis FKTP,Tipe FKTP,Tingkat Pelayanan FKTP,Jenis Poli FKTP,Segmen Peserta saat akses layanan FKTP,Kode dan nama diagnosis ICD 10 (3 digit),Jenis Kunjungan FKTP,Kelas Status Pulang Peserta,Dirujuk,Lama kunjungan FKTP
0,61,6108,3,1,4,1,12.0,2,9999,2,Sehat,0.0,0 days
1,51,5171,9,3,2,1,1.0,4,773,1,Belum_Sehat,0.0,0 days
2,35,3516,9,2,1,1,3.0,4,1757,1,Belum_Sehat,0.0,0 days
3,34,3402,3,1,4,1,1.0,5,621,1,Belum_Sehat,0.0,0 days
4,35,3509,3,1,4,1,1.0,1,622,1,Belum_Sehat,0.0,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4056893,35,3505,9,3,2,1,12.0,4,9999,2,Sehat,0.0,0 days
4056894,35,3514,9,2,1,1,1.0,5,561,1,Belum_Sehat,1.0,0 days
4056895,11,1171,4,2,1,1,13.0,5,9999,2,Sehat,0.0,0 days
4056896,74,7402,3,1,3,1,1.0,2,774,1,Belum_Sehat,0.0,0 days


## Preprocessing

In [12]:
#Encoding target variable

label_encoder = preprocessing.LabelEncoder()

train1['Kelas Status Pulang Peserta'] = label_encoder.fit_transform(train1['Kelas Status Pulang Peserta'])

train1['Jenis Kunjungan FKTP'] = label_encoder.fit_transform(train1['Jenis Kunjungan FKTP'])
test1['Jenis Kunjungan FKTP'] = label_encoder.fit_transform(test1['Jenis Kunjungan FKTP'])

In [13]:
encode_col = ['Provinsi FKTP', 'Kabupaten/Kota FKTP', 'Kepemilikan FKTP', 'Jenis FKTP', 'Tipe FKTP', 'Tingkat Pelayanan FKTP', 'Jenis Poli FKTP', 'Segmen Peserta saat akses layanan FKTP', 'Kode dan nama diagnosis ICD 10 (3 digit)']

for dataset in clean_data:
    for i in encode_col:
        top10 = dataset[i].value_counts().sort_values(ascending=False).head(10).index
        for label in top10:
            dataset[f'{i} {label}'] = np.where(dataset[i]==label, 1, 0)
    
    dataset.drop(encode_col, axis=1, inplace=True)

In [14]:
print(train1.shape)
print(test1.shape)

(4056898, 76)
(1014225, 75)


In [18]:
train_x = train1.copy()
train_x.drop('Kelas Status Pulang Peserta', axis=1, inplace=True)
train_x.to_numpy()
train_y = train1['Kelas Status Pulang Peserta'].copy()

In [16]:
print(np.unique(train_y,return_counts=True))

(array([0, 1]), array([2634511, 1422387], dtype=int64))


In [26]:
model_pipeline = imbpipeline([
    ('over', SMOTE(sampling_strategy=.15)),
    ('under', RandomUnderSampler(sampling_strategy=.5)),
    ('xgb', xgb.XGBClassifier(n_estimators=50, objective='binary:logistic', 
                              verbosity=2))
])

In [30]:
from skopt.space import Real, Integer, Categorical
from skopt import BayesSearchCV


In [31]:
params = {'over__sampling_strategy': Real(.1, .5, 'log-uniform'),
         'under__sampling_strategy': Real(.5, .8, 'log-uniform'),
         'xgb__learning_rate': Real(1e-2, 5e-1),
         'xgb__max_depth': Integer(6, 64),
         'xgb__subsample': Real(0.5, 1),
         'xgb__colsample_bytree': Real(0.5, 1),
         'xgb__reg_alpha': Real(1e-2, 10, 'log-uniform'),
         'xgb__reg_lambda': Real(1e-2, 10, 'log-uniform'),}

search = BayesSearchCV(model_pipeline, params, scoring='f1', cv=3, verbose=10, 
                       n_jobs=-1, n_iter=20, return_train_score=True)

In [33]:
search.fit(train_x, train_y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


TypeError: The DType <class 'numpy.dtype[timedelta64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[int64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[timedelta64]'>, <class 'numpy.dtype[int32]'>, <class 'numpy.dtype[int32]'>, <class 'numpy.dtype[int32]'>, <class 'numpy.dtype[int32]'>, <class 'numpy.dtype[int32]'>, <class 'numpy.dtype[int32]'>, <class 'numpy.dtype[int32]'>, <class 'numpy.dtype[int32]'>, <class 'numpy.dtype[int32]'>, <class 'numpy.dtype[int32]'>, <class 'numpy.dtype[int32]'>, <class 'numpy.dtype[int32]'>, <class 'numpy.dtype[int32]'>, <class 'numpy.dtype[int32]'>, <class 'numpy.dtype[int32]'>, <class 'numpy.dtype[int32]'>, <class 'numpy.dtype[int32]'>, <class 'numpy.dtype[int32]'>, <class 'numpy.dtype[int32]'>, <class 'numpy.dtype[int32]'>, <class 'numpy.dty

In [ ]:
search.best_params_, search.best_score_